In [16]:
import requests
from bs4 import BeautifulSoup

# from google.colab import files

url_req_ = (['?imei=350424060069364&s_dt=2022-08-10%2011:50:00&e_dt=2022-08-10%2015:00:00',
            '?imei=352094089278805&s_dt=2022-08-10%2011:50:00&e_dt=2022-08-10%2015:00:00'])

url_req = (['?imei=350424060069364&s_dt=2022-08-16%2008:00:00&e_dt=2022-08-16%2017:30:00',
            '?imei=359632102795827&s_dt=2022-08-16%2008:00:00&e_dt=2022-08-16%2017:30:00'])


web_link = 'https://nav.by/info/testAccel.php'

db_files = []

### postman
import requests

url = "https://nav.by/login_get_data.php?type=LOGIN&data%5Blogin%5D=diler_el_vek&data%5Bpwd%5D=111111&data%5BscreenWidth%5D=1920&data%5BscreenHeight%5D=1080"

payload = {}
headers = {
    'Cookie': 'pc_id=60b9bbc0-bef7-4c4b-9052-003672c5d1a9; ses_id=15fb4b00-046b-4e59-bd88-175381dd03b8'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


### postman

def remove_str(value, deletechars_):
    for c in deletechars_:
        value = value.replace(c, '_')
    return value


NUM_OF_FILES = len(url_req)
for p in range(NUM_OF_FILES):

    name_of_file = remove_str(url_req[p], '\/:*?"<>|&%=-')
    db_files.append(name_of_file)

    print(name_of_file)

    file = open(name_of_file + ".csv", "w")
    file.close()

    response = requests.request("GET", web_link + url_req[p], headers=headers, data=payload)
    soup = BeautifulSoup(response.text, 'lxml')

    file = open(name_of_file + ".csv", "a+")

    quotes = soup.find_all('p')
    for quote in quotes:
        file.write(quote.text)
    file.close()

    file = open(name_of_file + ".csv", "r")
    lines = file.readlines()
    file.close()

    f_first_line = 1

    file = open(name_of_file + ".csv", "w")
    for line in lines:
        if f_first_line == 1:
            file.write(line)
            f_first_line = 0
        if ((
                line != "date;speed;78;64011;64022;64033;64044;delta64011;delta64022;delta64033;delta64044;vodo;lat;lon;dir;alt;engine_speed;distance_gps;201;202;203;204;210;211;212;213;214;215;ain1;ain2;151\n") and (
                line != "\n")):
            file.write(line)

    file.close()

{"url":".\/info\/help.php","exitCode":0}
_imei_350424060069364_s_dt_2022_08_16_2008_00_00_e_dt_2022_08_16_2017_30_00
_imei_359632102795827_s_dt_2022_08_16_2008_00_00_e_dt_2022_08_16_2017_30_00


In [17]:
import pandas as pd
from datetime import datetime

df = pd.read_csv(db_files[0] + '.csv', sep=';', engine='python', decimal=',')
df.columns = ['date', 'speed', 'rowdata', 'z', 'y', 'x_m', 'x_p', 'delta64011', 'delta64022', 'delta64033',
              'delta64044', 'vodo', 'lat', 'lon', 'dir', 'alt', 'engine_speed', 'distance_gps', 'sum_ES',
              'mark_ind1', 'sum_ES_APP', 'mark_ind2', 'sum_ES_2_APP', 'mark_ind3', 'sum_ES_APP_2', 'mark_ind4',
              'sum_ES_2_APP_2', 'mark_ind5', 'AnIn1', 'AnIn2', 'CAN1']

df_ = pd.read_csv(db_files[1] + '.csv', sep=';', engine='python', decimal=',')
df_.columns = ['date', 'speed', 'rowdata', 'z', 'y', 'x_m', 'x_p', 'delta64011', 'delta64022', 'delta64033',
              'delta64044', 'vodo', 'lat', 'lon', 'dir', 'alt', 'engine_speed', 'distance_gps', 'sum_ES',
              'mark_ind1', 'sum_ES_APP', 'mark_ind2', 'sum_ES_2_APP', 'mark_ind3', 'sum_ES_APP_2', 'mark_ind4',
              'sum_ES_2_APP_2', 'mark_ind5', 'AnIn1', 'AnIn2', 'CAN1']

k = 0

for i in range(2, len(df)):
    while datetime.strptime(df.date[i], "%Y-%m-%d %H:%M:%S.%f") > datetime.strptime(df_.date[k], "%Y-%m-%d %H:%M:%S.%f"):
        k += 1
    df.at[i, 'sum_ES'] = df_.sum_ES[k]
    df.at[i, 'sum_ES_APP'] = df_.sum_ES_APP[k]
    df.at[i, 'sum_ES_2_APP'] = df_.sum_ES_2_APP[k]
    df.at[i, 'sum_ES_APP_2'] = df_.sum_ES_APP_2[k]
    df.at[i, 'sum_ES_2_APP_2'] = df_.sum_ES_2_APP_2[k]

df.to_csv(db_files[0] + '_changed' + '.csv', index=False)